In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectConfig
from loss_funcs import *
from algs.random_sampling import RandomSampling
from algs.particle_swarm import ParticleSwarm
from algs.simulated_annealing import SimulatedAnnealing
from algs.random_restarts import RandomRestarts
from algs.differential_evolution import DifferentialEvolution

from eval_funcs import *
from evaluator import Evaluator

# Create a camera configuration
cam_config = CameraConfig(position=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

loss_func = MSE()

alg1 = ParticleSwarm(sim_viewer, loss_func=loss_func)
alg1_config = ParticleSwarm.Config(time_limit=3, population=30, num_iters=300)

alg2 = SimulatedAnnealing(sim_viewer, loss_func=loss_func)
alg2_config = SimulatedAnnealing.Config(time_limit=3, num_iters=100, samples_per_temp=150)

alg3 = RandomSampling(sim_viewer, loss_func=loss_func)
alg3_config = RandomSampling.Config(time_limit=3, num_samples=10000)

alg4 = RandomRestarts(alg1)
alg4_config = RandomRestarts.Config(time_limit=50, num_restarts=5, inner_config=alg1_config)

alg5 = DifferentialEvolution(sim_viewer, loss_func=loss_func)
alg5_config = DifferentialEvolution.Config(time_limit=50, population=30, mut_prob=0.3)

evaluator = Evaluator(world_viewer, eval_func=IOU_Diff(method="mae"))

init_position = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
init_positions = [ObjectConfig(orient, init_position) for orient in random_orientations]

for alg, config in [(alg5, alg5_config)]:
    losses = evaluator.evaluate(alg, config, init_positions)
    print(f"{type(alg).__name__}: {losses}")

""" for orient in random_orientations:

    img_world, obj_config = world_viewer.get_view_cropped(ObjectConfig(orient, init_position), depth=True)
    print(img_world.shape)
    print(img_world.dtype)
    print(img_world.min(), img_world.max())

    img_world, obj_config = world_viewer.get_view_cropped(ObjectConfig(orient, init_position))

    best_orient = search_alg.find_orientation(
        ref_img=img_world, ref_position=obj_config.position, alg_config=ParticleSwarm.Config(time_limit=100)
    )

    best_image, _ = sim_viewer.get_view_cropped(ObjectConfig(best_orient, obj_config.position))

    cv.imshow("real_img", img_world)
    cv.imshow("sim_img", best_image)
    cv.waitKey(0) """

In [ ]:
cv.destroyAllWindows()